# Install required enviroment and import packages

In [ ]:
!pip install pymorphy2

In [ ]:
import pandas as pd
import regex as re
from collections import Counter
import sys
from unicodedata import category
import pymorphy2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


You should be already familiar with this function, haha

In [ ]:
def preproc_data(dataset):
  dataset = csv_deEmojify(dataset)
  try:
    dataset = dataset.rename(columns={"Заголовок + конец сообщения": "start_end", "Только заголовок": "start"})
    dataset = dataset[["start_end", "start"]]
  except:
    print("noup")
  
  return dataset

#Load data

In [ ]:
HR = pd.read_excel("/content/drive/MyDrive/course_work/Headings_wo_emoji.xlsx")
HR_headings = HR.start_end.values

In [ ]:
small_GPT_headings = pd.read_excel("/content/drive/MyDrive/course_work/work_with_ends/small_gpt.xlsx").rename(columns={0: "mess"})
small_GPT_headings = small_GPT_headings["mess"].values

In [ ]:
medium_GPT_headings = pd.read_excel("/content/drive/MyDrive/course_work/work_with_ends/medium_gpt.xlsx").rename(columns={0: "mess"})
medium_GPT_headings = medium_GPT_headings["mess"].values

In [ ]:
dungeon = pd.read_excel("/content/drive/MyDrive/course_work/work_with_ends/unfinetune_dungeon.xlsx").rename(columns={0: "mess"})
dungeon_headings = dungeon.mess.values

# Let's count some metrics manually instead of write parser for SEO-analysis in the internet (for instance)

### Metrics I'm gonna count:

1. An amount of symbols

2. An amount of words

3. An amount of unique words

4. A classic text nausea

5. An amount of symbols of punctuation

6. A repeat rate

7. A degree of water

8. An amount for stop-words (words, that don't have any meaning beyond a context, are named stop-words)

Function to count an amount of symbols

In [ ]:
punctuation_chars =  [chr(i) for i in range(sys.maxunicode) 
                             if category(chr(i)).startswith("P")]
def count_punc(string):
  count_punc = 0
  for symb in string:
    if symb in punctuation_chars:
      count_punc += 1
      
  return count_punc

Function to count an amount of stop-words (you must have a file with russian stop-words for it)

In [ ]:
stop_words = open("/content/drive/MyDrive/course_work/stop_words_ru.txt", "r").read()

def count_stop_words(string):
  n_stop_words = 0
  for word in string.split():
    if word in stop_words:
      n_stop_words += 1
  return n_stop_words

Function to a little bit preprocessing string for further metrcis exploring

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def preproc_string(string):
  string = string.lower()
  for symb in string:
    if symb == "-":
      string = string.replace(symb, " ")
    elif symb in punctuation_chars:
      string = string.replace(symb, "")
  
  return " ".join([morph.parse(word)[0][2] for word in string.split()])
  

In [ ]:
trash = ["-", "—", ":)", ")", ":", "(", "(:", "!", "?", ",", ".", ""]
pattern = r"[!?.]"
vowels = ["a", "у", "о", "и", "э", "е", "ё", "я", "ю", "ы", "a", "e", "u", "i", "o", "y"]

def count_words(string):
  words = list(filter(lambda x: x not in trash, string.split()))
  return len(words)

def count_sent(string):
  sentences = list(filter(lambda x: x not in trash, re.split(pattern, string)))
  return len(sentences)

def count_sylls(string):
  counter = 0
  for char in string.lower():
    if char in vowels:
      counter += 1
  return counter 

def count_flash(string):
  n_words = count_words(string)
  n_sents = count_sent(string)
  n_sylls = count_sylls(string)
  flash = 206.835 - 1.05*n_words/n_sents - 84.6*n_sylls/n_words
  if flash > 100:
    return 100
  if flash < 0:
    return 0
  return flash

In [ ]:
# изменим порядок

metrics = metrics[['Unnamed: 0', 'message', 'mess_len', 'n_words', 'n_unique',
                    'repeat_rate', 'puke', 'puke_rate', 'n_stop', 'water_rate',
                    'flash', 'n_punc', 'model']]

In [ ]:
metrics.to_excel("all_metrics.xlsx")

In [ ]:
# подсчитываем длину сообщения, количество слов, количество знаков препинания, часть воды
# количество уникальных слов, степень повторенности, классическую тошноту, количество стоп-слов

def count_metrics(string):
  n_chars = len(string)
  n_punc = count_punc(string)
  flash = count_flash(string)

  string = preproc_string(string)
  n_stop_words = count_stop_words(string)
  n_words = len(string.split())
  n_unique_words = len(list(set(string.split())))
  repeat_degree = round((n_words - n_unique_words)/n_words, 4)
  puke_degree = max(Counter(string.split()).values())**0.5
  water_degree = round(n_stop_words / n_unique_words, 4)

  return n_chars, n_words, n_unique_words, n_punc, repeat_degree, puke_degree/n_words, water_degree, flash

In [ ]:
metrics = ["n_chars", "n_words", "n_unique_words", "n_punc", "repeat_rate", "puke_rate", "water_rate", "flash"]

# Count metrcis for original HR messages

In [ ]:
HR_metrics = []
for mess in HR_headings:
  HR_metrics.append(count_metrics(mess))

In [ ]:
HR_metrics = pd.DataFrame(HR_metrics, columns=metrics)

# Count metrcis for generated GPT-2 messages

In [ ]:
small_gpt_metrics = []
for mess in small_GPT_headings:
  small_gpt_metrics.append(count_metrics(mess))

In [ ]:
small_gpt_metrics = pd.DataFrame(small_gpt_metrics, columns=metrics)

In [ ]:
medium_gpt_metrics = []
for mess in medium_GPT_headings:
  medium_gpt_metrics.append(count_metrics(mess))

In [ ]:
medium_gpt_metrics = pd.DataFrame(medium_gpt_metrics, columns=metrics)

# For unfinetuned Dungeon

In [ ]:
dungeon_metrics = []
for mess in dungeon_headings:
  dungeon_metrics.append(count_metrics(mess))

In [ ]:
dungeon_metrics = pd.DataFrame(dungeon_metrics, columns=metrics)

# Save all this data to an excel-files

In [ ]:
HR_metrics.to_excel("HR_metrics.xlsx", index=False)

In [ ]:
small_gpt_metrics.to_excel("small_gpt_metrics.xlsx", index=False)

In [ ]:
medium_gpt_metrics.to_excel("medium_gpt_metrics.xlsx", index=False)

In [ ]:
dungeon_metrics.to_excel("unfinetuned_dungeon_metrics.xlsx", index=False)